In [ ]:
#%matplotlib qt
%matplotlib inline
import os
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
import torchtext
import torch.nn as nn
import nltk
import pandas as pd
nltk.download('punkt')
cmap = plt.get_cmap('viridis')
from mpl_toolkits.mplot3d import Axes3D

In [2]:
[a for a in range(10) if a<5]

[0, 1, 2, 3, 4]

# Grid search

python -m grid results_grid "python automatic_data_generation/train_and_eval_cvae.py -ng 2000 -ep 50" --dataset-size:int 125 250 500 1000 --seed:int 1 2 3 --temperature:float 0.5 1 2 --embedding-dimension:int 100 300 --hidden-size:int 256 512 --n_parallel 8

In [84]:
from pathlib import Path
import shutil
results_dir = 'results_grid'
runs = []

for filename in os.listdir(results_dir):
    filepath = Path(results_dir)/Path(filename)
    if filename.endswith('.pkl'):
        runs.append(torch.load(filepath))
        
runs = [run for run in runs if (run['args']['dataset_size']==1000)]
        
data = {}
for arg_name in runs[0]['args'].keys():
    data[arg_name] = [run['args'][arg_name] for run in runs]
data['pickle']   = np.array([run['args']['pickle'].split('/')[-1].rstrip('.pkl') for run in runs])
    
data['bleu_quality']   = np.array([run['delexicalised_metrics']['bleu_scores']['quality']['avg'] for run in runs])
data['bleu_diversity'] = np.array([run['delexicalised_metrics']['bleu_scores']['diversity']['avg'] for run in runs])
data['accuracy']       = np.array([run['delexicalised_metrics']['intent_accuracy'] for run in runs])
data['originality']       = np.array([run['delexicalised_metrics']['originality']['avg'] for run in runs])
data['transfer']   = np.array([run['delexicalised_metrics']['transfer']['avg'] for run in runs])

data['metric'] = data['accuracy']**10 * data['originality'] #* data['bleu_quality']

df = pd.DataFrame(data, columns=['pickle', 'dataset_size', 'n_epochs', 'seed', #'x1', 'x2', 'k1', 'k2', 
                                 'hidden_size', 'embedding_dimension', 'temperature',
                                 'bleu_quality', 'bleu_diversity', 'accuracy', 'originality', 'transfer', 'metric'])
df = df.sort_values(by = 'metric', ascending=False)
print(len(runs))
df.head(100)

36


,pickle,dataset_size,n_epochs,seed,hidden_size,embedding_dimension,temperature,bleu_quality,bleu_diversity,accuracy,originality,transfer,metric
26,04810,1000,50,1,256,100,0.5,0.681846,0.103404,0.9830,0.564106,0.274489,4.752212e-01
2,27623,1000,50,2,256,100,0.5,0.530723,0.089179,0.9770,0.595338,0.293538,4.717472e-01
7,13167,1000,50,3,256,100,0.5,0.743735,0.043356,0.9940,0.408776,0.107256,3.849007e-01
33,34962,1000,50,3,256,300,0.5,0.727332,0.030813,0.9955,0.391477,0.101286,3.742126e-01
8,27771,1000,50,2,256,300,0.5,0.577924,0.033333,0.9915,0.377342,0.117269,3.464674e-01
1,44367,1000,50,2,256,300,1.0,0.597028,0.023854,0.9975,0.321214,0.054744,3.132730e-01
14,31117,1000,50,1,256,300,0.5,0.794566,0.027973,0.9915,0.326144,0.113074,2.994585e-01
13,20113,1000,50,2,256,300,2.0,0.615379,0.017703,1.0000,0.274585,0.005602,2.745854e-01
18,45346,1000,50,3,256,300,1.0,0.705314,0.038313,0.9965,0.276666,0.059240,2.671334e-01
22,24300,1000,50,1,256,300,1.0,0.729234,0.014911,0.9910,0.285455,0.049980,2.607800e-01


In [22]:
irun = np.argmax(data['metric'])
print(irun)
run = runs[irun]
list(zip(run['generated']['intents'], runs[irun]['generated']['utterances']))[:10]

4


[('BookRestaurant',
  'i want to book a reservation for can fabes  in brazil  for a party of 8  '),
 ('GetWeather',
  'is it going to be nicer 2 months from now  in lecanto  sierra leone  '),
 ('GetWeather', 'what is the weather forecast for lansford  pennsylvania  '),
 ('BookRestaurant',
  'i need to book a table at a delicatessen  that serves chichi  in lecanto  '),
 ('AddToPlaylist', 'add album  to my  retrowave outrun  '),
 ('SearchCreativeWork', 'find the the entity  album  '),
 ('PlayMusic', 'play any seventies  music on itunes  '),
 ('BookRestaurant',
  'lets go in the area  to the maisonette  in district of columbia  '),
 ('GetWeather', 'what will the weathr be in west richfield  maldives  '),
 ('GetWeather', 'i want to know the chillier  from 02:22 pm  in andorra  ')]

## Get all the seeds of the best run

In [85]:
# Get best runs (all seeds)

ibest = np.argmax(data['metric'])
iruns = [ibest]
best = runs[ibest]
best_args = [value for arg, value in best['args'].items() if arg not in ['seed','pickle']]

for irun, run in enumerate(runs):
    if irun==ibest:
        continue
    args = [value for arg, value in run['args'].items() if arg not in ['seed','pickle']]
    #print(args)
    if args == best_args:
        iruns.append(irun)

pickles = [runs[i]['args']['pickle'].rstrip('.pkl') for i in iruns]
print(pickles)

# Save selected runs

savedir = results_dir + '/selected_runs_{}'.format(best['args']['dataset_size'])
if os.path.exists(savedir):
    ! sudo rm -r {savedir} # shutil.rmtree(savedir)
! sudo mkdir {savedir}

for pickle in pickles:
    ! sudo cp -r {pickle} {savedir}
    ! sudo cp {pickle}.pkl {savedir}

print('done')

['results_grid/04810', 'results_grid/27623', 'results_grid/13167']
done
